# 7.1 Function Calling 原理实验

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/07-function-calling/function_calling.ipynb)

**预计 API 费用: ~$0.01**

本 Notebook 演示 Function Calling 的完整工作流程,包括天气查询、流式输出等。

In [ ]:
!pip install -q openai

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("请输入你的 OpenAI API Key: ")

## 实验 1: 基础 Function Calling

实现一个简单的天气查询工具。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

# 1. 定义工具
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的实时天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称,如:北京、上海、深圳"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "温度单位"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    }
]

# 2. 实现工具函数(模拟数据)
def get_weather(city: str, unit: str = "celsius"):
    """模拟天气 API(实际应用中调用真实 API)"""
    fake_data = {
        "北京": {"temperature": 15 if unit == "celsius" else 59, "weather": "晴", "humidity": 45},
        "上海": {"temperature": 22 if unit == "celsius" else 72, "weather": "多云", "humidity": 65},
        "深圳": {"temperature": 28 if unit == "celsius" else 82, "weather": "雨", "humidity": 80},
    }
    return fake_data.get(city, {"error": f"未找到城市 {city} 的天气数据"})

# 3. 测试单个工具调用
messages = [{"role": "user", "content": "北京今天天气怎么样?"}]

print("=== 第 1 步: 用户提问 ===")
print(messages[0]["content"])

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message
print(f"\n=== 第 2 步: LLM 判断需要调用工具 ===")

if message.tool_calls:
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    
    print(f"工具: {function_name}")
    print(f"参数: {json.dumps(function_args, ensure_ascii=False)}")
    
    # 4. 执行工具
    print(f"\n=== 第 3 步: 执行工具 ===")
    result = get_weather(**function_args)
    print(f"工具返回: {json.dumps(result, ensure_ascii=False)}")
    
    # 5. 将结果返回 LLM
    messages.append(message)
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result, ensure_ascii=False)
    })
    
    print(f"\n=== 第 4 步: LLM 生成最终回答 ===")
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    print(final_response.choices[0].message.content)
else:
    print("LLM 判断不需要调用工具,直接回答:")
    print(message.content)

## 实验 2: 多轮工具调用

用户可能连续问多个城市的天气。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

def run_conversation(user_query: str, max_iterations: int = 5):
    """运行多轮对话,自动处理工具调用"""
    messages = [{"role": "user", "content": user_query}]
    
    for i in range(max_iterations):
        print(f"\n--- 迭代 {i+1} ---")
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        messages.append(message)
        
        if not message.tool_calls:
            print("\n✅ 最终回答:")
            print(message.content)
            return message.content
        
        # 执行所有工具调用
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 调用工具: {function_name}({function_args})")
            
            result = get_weather(**function_args)
            print(f"📊 返回: {json.dumps(result, ensure_ascii=False)}")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result, ensure_ascii=False)
            })
    
    return "达到最大迭代次数"

# 测试
run_conversation("北京、上海、深圳今天天气如何?哪个最适合出门?")

## 实验 3: tool_choice 参数控制

控制 LLM 是否调用工具。

In [ ]:
from openai import OpenAI

client = OpenAI()

question = "你好"
messages = [{"role": "user", "content": question}]

# 1. auto: 自动判断
print("=== tool_choice='auto' ===")
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
print(f"是否调用工具: {bool(response.choices[0].message.tool_calls)}")
print(f"回答: {response.choices[0].message.content}\n")

# 2. none: 强制不调用工具
print("=== tool_choice='none' ===")
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "北京天气怎么样?"}],
    tools=tools,
    tool_choice="none"
)
print(f"是否调用工具: {bool(response.choices[0].message.tool_calls)}")
print(f"回答: {response.choices[0].message.content}\n")

# 3. required: 强制调用工具
print("=== tool_choice='required' ===")
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "你好"}],
    tools=tools,
    tool_choice="required"
)
print(f"是否调用工具: {bool(response.choices[0].message.tool_calls)}")
if response.choices[0].message.tool_calls:
    print(f"调用了: {response.choices[0].message.tool_calls[0].function.name}\n")

# 4. 指定工具
print("=== tool_choice={指定工具} ===")
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "今天适合出门吗?"}],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "get_weather"}}
)
print(f"是否调用工具: {bool(response.choices[0].message.tool_calls)}")
if response.choices[0].message.tool_calls:
    print(f"调用了: {response.choices[0].message.tool_calls[0].function.name}")

## 实验 4: 流式输出(Streaming)

Function Calling 也支持流式输出。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

messages = [{"role": "user", "content": "北京天气如何?"}]

print("=== 流式输出工具调用 ===\n")

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    stream=True
)

tool_calls_buffer = []
current_tool_call = None

for chunk in stream:
    delta = chunk.choices[0].delta
    
    if delta.tool_calls:
        for tool_call_chunk in delta.tool_calls:
            index = tool_call_chunk.index
            
            # 初始化工具调用
            if len(tool_calls_buffer) <= index:
                tool_calls_buffer.append({
                    "id": "",
                    "function": {"name": "", "arguments": ""}
                })
            
            # 累积数据
            if tool_call_chunk.id:
                tool_calls_buffer[index]["id"] = tool_call_chunk.id
            if tool_call_chunk.function.name:
                tool_calls_buffer[index]["function"]["name"] = tool_call_chunk.function.name
                print(f"🔧 调用工具: {tool_call_chunk.function.name}")
            if tool_call_chunk.function.arguments:
                tool_calls_buffer[index]["function"]["arguments"] += tool_call_chunk.function.arguments
                print(f"   参数片段: {tool_call_chunk.function.arguments}")

print("\n=== 完整工具调用 ===")
for tool_call in tool_calls_buffer:
    print(f"ID: {tool_call['id']}")
    print(f"函数: {tool_call['function']['name']}")
    print(f"参数: {tool_call['function']['arguments']}")

## 实验 5: 多个工具组合

定义多个工具,LLM 自动选择合适的工具。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

# 定义多个工具
multi_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取城市的实时天气",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "城市名"}
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "执行数学计算",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "数学表达式,如 '2+3*4'"}
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "获取当前时间",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {"type": "string", "description": "时区,如 'Asia/Shanghai'"}
                },
                "required": []
            }
        }
    }
]

# 实现工具函数
def calculator(expression: str):
    try:
        result = eval(expression)
        return {"expression": expression, "result": result}
    except:
        return {"error": "计算错误"}

def get_time(timezone: str = "Asia/Shanghai"):
    from datetime import datetime
    return {"time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "timezone": timezone}

# 工具路由
available_functions = {
    "get_weather": get_weather,
    "calculator": calculator,
    "get_time": get_time
}

# 测试不同问题
test_questions = [
    "北京天气怎么样?",
    "帮我算一下 123 * 456",
    "现在几点了?",
    "北京天气如何?顺便告诉我现在几点"
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"问题: {question}")
    print(f"{'='*60}")
    
    messages = [{"role": "user", "content": question}]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=multi_tools
    )
    
    message = response.choices[0].message
    
    if message.tool_calls:
        messages.append(message)
        
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"\n🔧 调用: {function_name}({function_args})")
            
            func = available_functions[function_name]
            result = func(**function_args)
            
            print(f"📊 结果: {json.dumps(result, ensure_ascii=False)}")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result, ensure_ascii=False)
            })
        
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        
        print(f"\n✅ 最终回答:")
        print(final_response.choices[0].message.content)
    else:
        print(f"\n直接回答: {message.content}")

## 动手练习

1. **添加新工具**: 实现一个 `search_web(query)` 工具(模拟搜索)
2. **错误处理**: 当工具调用失败时,如何告诉 LLM?
3. **参数验证**: 在执行工具前验证参数是否合法
4. **成本统计**: 记录每次对话消耗的 tokens 和成本

---

## 关键要点总结

1. **Function Calling 是协作模式**: LLM 提出调用请求,你的代码执行
2. **工具定义用 JSON Schema**: 描述清楚参数类型和说明
3. **tool_choice 控制调用策略**: auto | required | none | 指定工具
4. **支持流式输出**: 实时接收工具调用信息
5. **多工具自动选择**: LLM 根据问题自动选择合适的工具

---

**下一步**: 学习 [7.2 Tool Use 实战](./tool_use.ipynb),对比 OpenAI 和 Anthropic 的工具调用方式。